In [4]:
import jax
import jax.numpy as np
jax.config.update('jax_enable_x64', True)
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
import schrodinger

## Introduction

This repository is built to simulate the Schrödinger equation. Provided a potential $V(t, x)$, moving boundaries $[x_1(t),x_2(t)]$, and boundary conditions $f_1(t)$, $f_2(t)$, a solution wavefunction $\psi$ satisfies

$$
\begin{aligned}
&i\partial_t \psi(t, x) = \left(-\frac{1}{2}\partial_x^2+V(t, x)\right)\psi(t, x)\\
&\begin{cases}
\psi\bigr\vert_{x_1}=f_1(t)\\[.5em]
\psi\bigr\vert_{x_2}=f_2(t)
\end{cases}
\end{aligned}
$$

or, alternatively, the same equation with periodic boundary conditions.

To build the problem we'll define functions for $x_1(t),x_2(t),f_1(t),f_2(t),V(t, x)$ and feed them to our numerical solver for later analysis and visualisation. This will return a `Wavefunction` object which we'll look at briefly. It contains information about a wavefunction and its energy.

Let's define some example data, bearing in mind that a `Wavefunction` normalises its data at each time step &mdash; you need not input a normalised amplitude.

In [5]:
Tmax = 2
L = 1
t = np.linspace(0, Tmax, 200)
x = np.linspace(-L/2, L/2, 300)
x1 = lambda t: -L/2
x2 = lambda t: L/2 
T, X = np.meshgrid(t, x, indexing='ij')
amplitude = np.cos(2*np.pi*X/L) * np.exp(1j*np.pi*(X/L-T/Tmax))
E = np.zeros(len(t))
dE = np.zeros(len(t))
psi = schrodinger.Wavefunction(amplitude, t, x1, x2, E, dE)

The `Wavefunction`object has two methods avaliable:
 - `plot`, which shows a 3D view of the wavefunction. Optional arguments for `t` and `x` plot the wavefunction at different instants of time or space.
 - `animate`, which returns an animation of the wavefunction.

In [6]:
psi.plot()
plt.show()

In [7]:
psi.plot(t=1)
plt.show()

In [8]:
psi.plot(x=.2)
plt.show()

In [9]:
ani = psi.animate()
plt.show()

With this covered, let's check out the eigenfunctions of well known systems: the particle in a box and the harmonic oscillator. We'll do this using the `solve` function, which requires:
- An initial wavefunction
- Times array
- Left boundary 
- Right boundary
- Potential
- Boundary condition type
- Boundary functions (optional)

In [10]:
# Particle in a box

L = 1
n = 3
k = n * np.pi / L
w = k**2 / 2
T = 4 / w

def PIB(n, x):
    L = x[-1] - x[0]
    k = n * np.pi / L 
    if n%2:
        return np.cos(k*x)
    else:
        return np.sin(k*x)

t = np.linspace(0, T, 100)
x = np.linspace(-L/2, L/2, 1000)
psi0 = PIB(n, x)
V  = lambda t, x: 0
x1 = lambda t: -L/2
x2 = lambda t: L/2 
psi = schrodinger.solve(psi0, t, x1, x2, V, BC='dirichlet')

In [11]:
psi.plot(t=0)
plt.show()

In [12]:
psi.plot(x=0)
plt.show()

In [13]:
psi.plot()
plt.show()

In [14]:
ani = psi.animate()
plt.show()

In [15]:
# Harmonic oscillator
from scipy.special import hermite

w = 1
n = 3
L = 6 * np.sqrt((2*n+1)/w)
T = 4 / w

def HO(n, x):
    Hn = hermite(n)
    return np.exp(-w/2*x**2) * Hn(np.sqrt(w)*x)

t = np.linspace(0, T, 100)
x = np.linspace(-L/2, L/2, 1000)
psi0 = HO(n, x)
V  = lambda t, x: w**2/2 * x**2
x1 = lambda t: -L/2
x2 = lambda t: L/2 
psi = schrodinger.solve(psi0, t, x1, x2, V, BC='periodic')

In [16]:
psi.plot()
plt.show()

In [17]:
ani = psi.animate()
plt.show()

Now that we have covered some basic examples, why not play around with moving boundaries and arbitrary boundary conditions?

In [18]:
L = 1
n = 3
k = n * np.pi / L
w = k**2 / 2
T = 4 / w

t = np.linspace(0, T, 100)
x = np.linspace(-L/2, L/2, 1000)
psi0 = PIB(n, x)
V  = lambda t, x: 0
x1 = lambda t: -L/2 - .05 * w * t
x2 = lambda t: L/2  + .05 * w * t
psi = schrodinger.solve(psi0, t, x1, x2, V, BC='dirichlet')

In [19]:
ani = psi.animate()
plt.show()

In [20]:
L = 1
T = 2

t = np.linspace(0, T, 100)
x = np.linspace(-L/2, L/2, 1000)
psi0 = 5* (x-L/2)*(x+L/2)**3 + 1
V  = lambda t, x: 0
x1 = lambda t: -1/2 - .05 * t
x2 = lambda t: 1/2 + .05  * t 
f1 = lambda t: 1
f2 = lambda t: 1
psi = schrodinger.solve(psi0, t, x1, x2, V, BC='dirichlet', f=[f1, f2])

In [21]:
ani = psi.animate()
plt.show()